This notebook queries Twitter with a simple query and stores results in a json file. 

In [7]:
!pip3 install searchtweets twarc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 7.1 MB/s eta 0:00:00
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34528 sha256=5b411daf94b051837420c8d682af0ff8565ee07241f8594bd40d1ca24b60d83f
  Stored in directory: /Users/katyaartemova/Library/Caches/pip/wheels/90/cd/58/816b023d3eb0e2c9c6e0cd834a10a53e932e29d7c719298216
Successfully built configobj

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [12]:
from twarc import Twarc2, expansions
import datetime
import json

bearer_token = open('bearer_token.txt').readlines()[0].strip()

client = Twarc2(bearer_token=bearer_token)


def main(query):
    
    tweets = []

    start_time = datetime.datetime(2021, 1, 1, 0, 0, 0, 0, datetime.timezone.utc)
    end_time = datetime.datetime(2021, 5, 30, 0, 0, 0, 0, datetime.timezone.utc)
    

    search_results = client.search_all(query=query, start_time=start_time, end_time=end_time, max_results=10)

    for page in search_results:

        result = expansions.flatten(page)
        for tweet in result:
            tweets.append(json.dumps(tweet))
    return tweets


In [14]:
query = "gscheid"
tweets = main(query)

rate limit exceeded: sleeping 437.8378858566284 secs


In [18]:
with open(f'tweets/tweets_{query}.txt','w+') as outfile:
    for tweet in tweets:
        outfile.write(tweet)
          
